In [ ]:
using DataFrames
using PyPlot
using Statistics
using Juliana

# Config

In [ ]:
work_dir = "output_dose-calc_at_ct_res"
mkpath(work_dir);

In [ ]:
patient_ID = "test_00"
data_dir = "/data/user/bellotti_r/data";

plan_file = "/data/user/bellotti_r/differentiable-planning/output/clinical_BA/test_00/result_plan.json";
original_dose_file = "/data/user/bellotti_r/differentiable-planning/output/clinical_BA/test_00/result_dose.dat"

In [ ]:
fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.8.jar";

# Load data

In [ ]:
ct_path, patient_data = Juliana.load_patient_data(data_dir, patient_ID);

In [ ]:
target_name, target_dose = Juliana.hottest_target(patient_data.prescriptions);
target = patient_data.structures[target_name];

In [ ]:
plan = Juliana.FionaStandalone.read_plan_file(plan_file);

In [ ]:
weights = vcat([DataFrame(f.spots).weight for f in plan.fields]...);

In [ ]:
original_dose = Juliana.load_dose_dat_file(original_dose_file);

# Calculate the dose at CT resolution using the Dij matrix

In [ ]:
Dij = Juliana.FionaStandalone.calculate_dose_on_grid(
    plan,
    patient_data.ct.grid,
    ct_path,
    target,
    target_dose,
    work_dir,
    fiona_standalone_bin_path,
    fiona_jar_path,
)

In [ ]:
dose = reshape(Dij * weights, patient_data.ct.grid.size...);

In [ ]:
Juliana.plot_distributions(
    patient_data.ct,
    [("original", original_dose.data), ("on grid", dose)],
    [["PTV", (patient_data.structures[target_name], "red")]],
    60, 180,
    60, 180,
    40,
    target_dose,
    Juliana.build_colorscheme(),
)

In [ ]:
abs_err = vec(abs.(dose .- original_dose.data));

In [ ]:
fig, ax = subplots()

ax.hist(abs_err, bins=100);

In [ ]:
quantile(abs_err, [0.95, 0.99])